## Import dependencies

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import os

from pandas.io import sql

from pandas import DataFrame


import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode

In [2]:
from config import database_username
from config import database_password

### Read all the file names and store them into a List

In [3]:
pwd = os.getcwd()
#print(pwd)
pwd = pwd + '/data/csv'
#print(pwd)


for path, folders, files in os.walk(pwd):
   ## just rename folders inside path 
    if pwd == path:
        #print(path)
        #print(folders)
        print(files)
print(type(files))
print(len(files))

['201709-citibike-tripdata.csv', '201809-citibike-tripdata.csv', '201701-citibike-tripdata.csv', '201602-citibike-tripdata.csv', '201902-citibike-tripdata.csv', '201512-citibike-tripdata.csv', '201411-citibike-tripdata.csv', '201801-citibike-tripdata.csv', '201407-citibike-tripdata.csv', '201504-citibike-tripdata.csv', '201309-citibike-tripdata.csv', '201608-citibike-tripdata.csv', '201510-citibike-tripdata.csv', '201803-citibike-tripdata.csv', '201405-citibike-tripdata.csv', '201506-citibike-tripdata.csv', '201703-citibike-tripdata.csv', '201311-citibike-tripdata.csv', '201610-citibike-tripdata.csv', '201606-citibike-tripdata.csv', '201307-citibike-tripdata.csv', '201705-citibike-tripdata.csv', '201805-citibike-tripdata.csv', '201403-citibike-tripdata.csv', '201508-citibike-tripdata.csv', '201807-citibike-tripdata.csv', '201502-citibike-tripdata.csv', '201811-citibike-tripdata.csv', '201401-citibike-tripdata.csv', '201711-citibike-tripdata.csv', '201612-citibike-tripdata.csv', '201604

In [4]:
# SOrt all files

files.sort()

### Connect to the database

In [5]:
database_ip       = '127.0.0.1'
database_name     = 'citybike_db'

In [6]:
mydb = mysql.connector.connect(
  host=database_ip,
  user=database_username,
  passwd=database_password
)

mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE IF NOT EXISTS citybike_db;")

mydb.close()

In [7]:
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(database_username, database_password, database_ip, database_name))

### Run the SQL file to create the table where the data will be stored

In [8]:
# Open and read the file as a single buffer
fd = open('create_db_schema.sql', 'r')
sqlFile = fd.read()
fd.close()

# all SQL commands (split on ';')
sqlCommands = sqlFile.split(';')


try:
    print('[#########] Running the create_db_schema.sql script')
    
    conn = mysql.connector.connect(host=database_ip,
                             database=database_name,
                             user=database_username,
                             password=database_password)
    cursor = conn.cursor()
    
    for command in sqlCommands:
        try:
            cursor.execute(command)
            #print(command)
        except Exception as e:
            print(f'Error detected: ', str(e))
            
    print('[#########] create_db_schema.sql script ran succesfuly')
    
except Exception as e:
        print(f'Error detected: ', str(e))
finally:
    #closing database connection.
    if(conn.is_connected()):
        conn.close()
        print('[#########] MySQL connection is closed')

[#########] Running the create_db_schema.sql script
Error detected:  1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '--drop table IF EXISTS d_trips' at line 1
Error detected:  1050 (42S01): Table 'd_trips' already exists
Error detected:  1065 (42000): Query was empty
[#########] create_db_schema.sql script ran succesfuly
[#########] MySQL connection is closed


### Load file by file to the database

In [12]:
for file in files:
    

    tripdata_file = file
    
    print(f'[#########] Loading {tripdata_file}....')
    
    df = pd.read_csv("data/csv/"+ tripdata_file)
    
    df.columns = ['tripduration','starttime','stoptime','start_station_id','start_station_name','start_station_latitude','start_station_longitude','end_station_id','end_station_name','end_station_latitude','end_station_longitude','bikeid','usertype','birth_year','gender']
    
    df['starttime'] = pd.to_datetime(df['starttime'])
    df['stoptime'] = pd.to_datetime(df['stoptime'])

    
    #df.columns = ['tripduration','starttime','stoptime','start_station_id','start_station_name','start_station_latitude','start_station_longitude','end_station_id','end_station_name','end_station_latitude','end_station_longitude','bikeid','usertype','birth_year','gender']

    
    ### Se guardan los datos de 10K en 10K en la base de datos
    ### porque por alguna razon si se meten mas datos el programa crashea

    cargar_de = 10000 

    indexx = 0
    limit = len(df)

    while (indexx<=limit):

        final =  indexx + cargar_de

        try:
            #print(f'[#########] Saving from index {indexx} to {final}.')
            df.iloc[indexx:final].to_sql(name='d_trips', con=database_connection, if_exists='append', index=False) 

        except Exception as e:
            print(f'Error detected in file', str(e))

        indexx = indexx + cargar_de

    print(f'[#########]  Data saved.')

[#########] Loading 201306-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201307-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201308-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201309-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201310-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201311-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201312-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201401-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201402-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201403-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201404-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201405-citibike-tripdata.csv....
[#########]  Data saved.
[#########] Loading 201406-citibike-tripdata.csv....
[#########]